<a href="https://colab.research.google.com/github/dpcks/playdata/blob/main/data_processing/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 범주형에 대한 가능한 인코딩 
    - 원핫 인코딩
    - 카운트 인코딩
    - 순서형 인코딩
- 모델은 다음의 모델만 사용
    - LogisticRegression
- 스케일링은 minmax 만 사용
    - MinMaxScaler
- 모든 랜덤 요소에 seed 값은 42
    - 검증셋 사이즈는 0.2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split # 검증셋 사이즈 0.2
from sklearn.linear_model import LogisticRegression # 모델
from sklearn.preprocessing import MinMaxScaler # 이걸로만 스케일링 할것
from sklearn.preprocessing import OneHotEncoder 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/data/"
SEED = 42

In [ ]:
df = pd.read_csv(f"{DATA_PATH}titanic.csv")
df

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  1309 non-null   int64  
 1   survived     1309 non-null   int64  
 2   pclass       1309 non-null   int64  
 3   name         1309 non-null   object 
 4   gender       1309 non-null   object 
 5   age          1046 non-null   float64
 6   sibsp        1309 non-null   int64  
 7   parch        1309 non-null   int64  
 8   ticket       1309 non-null   object 
 9   fare         1308 non-null   float64
 10  cabin        295 non-null    object 
 11  embarked     1307 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 122.8+ KB


In [ ]:
df["gender"] = df["gender"].map({"male":1,"female":0})

In [ ]:
cols = ["pclass","gender","age","sibsp","parch","fare"]
features = df[cols]
features.head()

,pclass,gender,age,sibsp,parch,fare
0,3,1,22.0,1,0,7.2500
1,1,0,38.0,1,0,71.2833
2,3,0,26.0,0,0,7.9250
3,1,0,35.0,1,0,53.1000
4,3,1,35.0,0,0,8.0500


In [ ]:
features["family"]= features["sibsp"] + features["parch"]
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean,fare_mean,age_mean,...,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,cabin_nan,embarked_C,embarked_Q,embarked_S,embarked_nan
0,3,1,22.0,1,0,7.2500,1,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,0,38.0,1,0,71.2833,1,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,0,26.0,0,0,7.9250,0,46.198097,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1,0,35.0,1,0,53.1000,1,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3,1,35.0,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,-1.0,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1305,1,0,39.0,0,0,108.9000,0,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1306,3,1,38.5,0,0,7.2500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1307,3,1,-1.0,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
agg_list=[
    ("gender+fare_mean",lambda x:x.mean())
]
tmp = df.groupby("gender")["fare"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean
0,3,1,22.0,1,0,7.2500,1,26.154601
1,1,0,38.0,1,0,71.2833,1,46.198097
2,3,0,26.0,0,0,7.9250,0,46.198097
3,1,0,35.0,1,0,53.1000,1,46.198097
4,3,1,35.0,0,0,8.0500,0,26.154601
...,...,...,...,...,...,...,...,...
1304,3,1,NaN,0,0,8.0500,0,26.154601
1305,1,0,39.0,0,0,108.9000,0,46.198097
1306,3,1,38.5,0,0,7.2500,0,26.154601
1307,3,1,NaN,0,0,8.0500,0,26.154601


- pclass 별 운임료에 대한 평균

In [ ]:
agg_list=[
    ("fare_mean",lambda x:x.mean())
]
tmp =df.groupby("pclass")["fare"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean,fare_mean
0,3,1,22.0,1,0,7.2500,1,26.154601,13.302889
1,1,0,38.0,1,0,71.2833,1,46.198097,87.508992
2,3,0,26.0,0,0,7.9250,0,46.198097,13.302889
3,1,0,35.0,1,0,53.1000,1,46.198097,87.508992
4,3,1,35.0,0,0,8.0500,0,26.154601,13.302889
...,...,...,...,...,...,...,...,...,...
1304,3,1,NaN,0,0,8.0500,0,26.154601,13.302889
1305,1,0,39.0,0,0,108.9000,0,46.198097,87.508992
1306,3,1,38.5,0,0,7.2500,0,26.154601,13.302889
1307,3,1,NaN,0,0,8.0500,0,26.154601,13.302889


In [ ]:
agg_list=[
    ("age_mean",lambda x:x.mean())
]
tmp = df.groupby("pclass")["age"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean,fare_mean,age_mean
0,3,1,22.0,1,0,7.2500,1,26.154601,13.302889,24.816367
1,1,0,38.0,1,0,71.2833,1,46.198097,87.508992,39.159930
2,3,0,26.0,0,0,7.9250,0,46.198097,13.302889,24.816367
3,1,0,35.0,1,0,53.1000,1,46.198097,87.508992,39.159930
4,3,1,35.0,0,0,8.0500,0,26.154601,13.302889,24.816367
...,...,...,...,...,...,...,...,...,...,...
1304,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367
1305,1,0,39.0,0,0,108.9000,0,46.198097,87.508992,39.159930
1306,3,1,38.5,0,0,7.2500,0,26.154601,13.302889,24.816367
1307,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367


- 나이별 운임료의 평균

In [ ]:
agg_list=[
    ("fare_mean",lambda x:x.mean())
]
tmp = df.groupby("age")["fare"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean,fare_mean,age_mean
0,3,1,22.0,1,0,7.2500,1,26.154601,13.302889,24.816367
1,1,0,38.0,1,0,71.2833,1,46.198097,87.508992,39.159930
2,3,0,26.0,0,0,7.9250,0,46.198097,13.302889,24.816367
3,1,0,35.0,1,0,53.1000,1,46.198097,87.508992,39.159930
4,3,1,35.0,0,0,8.0500,0,26.154601,13.302889,24.816367
...,...,...,...,...,...,...,...,...,...,...
1304,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367
1305,1,0,39.0,0,0,108.9000,0,46.198097,87.508992,39.159930
1306,3,1,38.5,0,0,7.2500,0,26.154601,13.302889,24.816367
1307,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367


In [ ]:
oh_cols = ["cabin","embarked"]
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(df[oh_cols]).toarray(),
    columns = enc.get_feature_names_out()
)

features = pd.concat([features,tmp],axis=1)
features

,pclass,gender,age,sibsp,parch,fare,family,gender+fare_mean,fare_mean,age_mean,...,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,cabin_nan,embarked_C,embarked_Q,embarked_S,embarked_nan
0,3,1,22.0,1,0,7.2500,1,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,0,38.0,1,0,71.2833,1,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,0,26.0,0,0,7.9250,0,46.198097,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1,0,35.0,1,0,53.1000,1,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3,1,35.0,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1305,1,0,39.0,0,0,108.9000,0,46.198097,87.508992,39.159930,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1306,3,1,38.5,0,0,7.2500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1307,3,1,NaN,0,0,8.0500,0,26.154601,13.302889,24.816367,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
features.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 201 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    pclass                 1309 non-null   int64  
 1    gender                 1309 non-null   int64  
 2    age                    1046 non-null   float64
 3    sibsp                  1309 non-null   int64  
 4    parch                  1309 non-null   int64  
 5    fare                   1308 non-null   float64
 6    family                 1309 non-null   int64  
 7    gender+fare_mean       1309 non-null   float64
 8    fare_mean              1309 non-null   float64
 9    age_mean               1309 non-null   float64
 10   cabin_A10              1309 non-null   float64
 11   cabin_A11              1309 non-null   float64
 12   cabin_A14              1309 non-null   float64
 13   cabin_A16              1309 non-null   float64
 14   cabin_A18              1309 non-null  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [ ]:
features= features.fillna(-1)
features.isnull().sum().sum()

0

In [ ]:
df.isnull().sum()

passengerid       0
survived          0
pclass            0
name              0
gender            0
age             263
sibsp             0
parch             0
ticket            0
fare              1
cabin          1014
embarked          2
dtype: int64

In [ ]:
df["age"]=df["age"].fillna(df["age"].std())
df["fare"]=df["fare"].fillna(df["fare"].std())
df.isnull().sum().sum()

1016

In [ ]:
scaler = MinMaxScaler()
x_data = scaler.fit_transform(features)
y_data = df["survived"]
x_data , y_data

(array([[1.        , 1.        , 0.28395062, ..., 0.        , 1.        ,
         0.        ],
        [0.        , 0.        , 0.48148148, ..., 0.        , 0.        ,
         0.        ],
        [1.        , 0.        , 0.33333333, ..., 0.        , 1.        ,
         0.        ],
        ...,
        [1.        , 1.        , 0.48765432, ..., 0.        , 1.        ,
         0.        ],
        [1.        , 1.        , 0.        , ..., 0.        , 1.        ,
         0.        ],
        [1.        , 1.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]), 0       0
 1       1
 2       1
 3       1
 4       0
        ..
 1304    0
 1305    1
 1306    0
 1307    0
 1308    0
 Name: survived, Length: 1309, dtype: int64)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2,random_state=SEED)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1047, 201), (262, 201), (1047,), (262,))

In [ ]:
model = LogisticRegression(random_state=SEED)
model.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [ ]:
pred = model.predict(x_valid)
accuracy_score(y_valid,pred)

0.8473282442748091

---------------------------------------------------------------------------------

In [ ]:
# df.select_dtypes("object").nunique()

In [ ]:
# tmp = df["cabin"].value_counts().reset_index()
# tmp

In [ ]:
# tmp.columns = ["cabin","cabin_cnt"]
# tmp.head()

In [ ]:
# features["cabin"]

In [ ]:
# features.merge(tmp,how="left", on="name").drop(columns=["name"])